In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
import spacy
import tensorflow as tf
import tensorflow_text as text
from transformers import BertTokenizer

nlp = spacy.load("en_core_web_lg")

In [ ]:
summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
test_summary = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')

In [ ]:
from string import punctuation as punc
from collections import defaultdict
import spacy
from string import punctuation as punc
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import spacy
from textblob import TextBlob
import math
from collections import Counter
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

def extract_lemmas(text):
    doc = nlp(text)
    lemmas = []

    for token in doc:
        if not token.is_stop:
            lemmas.append(token.lemma_.lower())
    return ' '.join(lemmas)


def count_punc(text):
    return len([p for p in text if p in punc])

# Overlap Score

def tree_depth_from_token(token):
    """Calculate the depth of a subtree rooted at a token."""
    if not list(token.children):
        return 1
    else:
        return 1 + max(tree_depth_from_token(child) for child in token.children)

def average_dependency_tree_depth(doc):
    """Compute the average depth of the dependency tree."""
    root_tokens = [tok for tok in doc if tok.dep_ == 'ROOT']
    if not root_tokens:
        return 0
    depths = [tree_depth_from_token(token) for token in root_tokens]
    return sum(depths) / len(depths)

def process_text(doc):
    stops = 0
    num_nouns = 0
    num_verbs = 0
    num_adverbs = 0
    
    for token in doc:
        if token.is_stop:
            stops += 1
        else:
            if token.pos_ == "VERB":
                num_verbs += 1
            elif token.pos_ == "NOUN":
                num_nouns += 1
            elif token.pos_ == "ADV":
                num_adverbs += 1
                
    return stops, num_nouns, num_verbs, num_adverbs

def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

def extract_features(row):
    doc = nlp(row['text'])
    context = nlp(row['prompt_text'])
    tokens = word_tokenize(row['text'].lower())
    context_tokens = word_tokenize(row['prompt_text'].lower())
    capital_error = sum(1 for sent in doc.sents if not sent.text[0].isupper())
    
    # 1. Length of the summary
    length = len(tokens)
    
    # 2. Number of unique words
    unique_words = len(set(tokens))
    
    #3. counter
    total_tokens = len(tokens) + 1e-11
    word_counts = Counter(tokens)
    once = sum(1 for word, count in word_counts.items() if count == 1)
    twice = sum(1 for word, count in word_counts.items() if count == 2)
    
    # 4. Named entities
    entities = set([ent.text.lower() for ent in doc.ents])
    context_entities = set([ent.text.lower() for ent in context.ents])
    num_entities = len(entities)
    context_num_entities = len(context_entities)
    
    # Calculate the overlap between the two sets
    overlap_entities = context_entities.intersection(entities)
    
    # Calculate the coverage score: (number of overlapping entities) / (number of entities in original text)
    coverage_score = len(overlap_entities) / len(context_entities) if len(context_entities) > 0 else 0
    num_overlap_entities = len(overlap_entities)
    
    named_entity_ratio = float(num_entities/total_tokens)
    context_entity_ratio = float(context_num_entities/len(context_tokens))
    
    # 5. Average word length
    avg_word_len = round(sum([len(token) for token in tokens])/total_tokens, ndigits=4)
    context_avg_word_len = round(sum([len(token) for token in context_tokens])/len(context_tokens), ndigits=4)
    
    # 6. Summary Polarity
    text_blob = TextBlob(row['text'])
    context_blob = TextBlob(row['prompt_text'])
    sentiment_polarity = text_blob.sentiment.polarity
    subjectivity = text_blob.subjectivity
    context_subjectivity = context_blob.subjectivity
    context_polarity = context_blob.sentiment.polarity
    error = sum(1 for token in tokens if token not in english)
    
    
    # 7. Stopwords verbs and nouns
    stops,num_nouns, num_verbs, num_adverbs = process_text(doc)
    context_stops,context_nouns, context_verbs, context_adverbs = process_text(context)
    
    # 8. Numerical entities
    num_numerical_entities = len([ent for ent in doc.ents if ent.label_ == "CARDINAL"])
    
    
    # 9. Sentece count
    sentences = [sent.text for sent in doc.sents]
    sentence = len(sentences) + 1e-12
    avg_sentence = float(sentence / total_tokens)
    avg_unique_sentence = float(sentence /unique_words) if unique_words > 0 else 0.0
    
    avg_sentence_length = sum(len(sent) for sent in sentences) / sentence
    max_sentence_length = max(len(sent) for sent in sentences)
    min_sentence_length = min(len(sent) for sent in sentences)
    std_sentence_length = (sum((len(sent) - avg_sentence_length)**2 for sent in sentences) / sentence)**0.5
    
    # 10 Tree Depth
    average_tree_depth = average_dependency_tree_depth(doc)
    prob_word = [count/total_tokens for word, count in word_counts.items()]
    results = {
        "nsubj": 0,
        "amod": 0,
        "advmod": 0,
        "xcomp": 0,
        "acomp": 0,
        "past": 0,
        "present": 0
    }
    
    num_syllables = 0
    
    for token in doc:
        if "VERB" in token.pos_:
            if "VBD" in token.tag_ or "VBN" in token.tag_:
                results["past"] += 1
            elif "VBG" in token.tag_ or "VBP" in token.tag_ or "VBZ" in token.tag_:
                results["present"] += 1
        # Check if the token's dependency is one of the desired dependencies
        if token.dep_ in results:
            results[token.dep_] += 1
            
        if token.is_alpha:
            num_syllables += syllable_count(token.text.lower())
    
    
    # Unigram similarity
    summary_set = set(tokens) if length > 0 else set()
    context_set = set(context_tokens) if len(context_tokens) > 0 else set()
    intersection = len(context_set.intersection(summary_set))
    difference = len(context_set.difference(summary_set))
    union = len(context_set.union(summary_set))
    precision = float(intersection /unique_words) if unique_words > 0 else 0.0
    recall = float(intersection / len(context_set)) if len(context_set) > 0 else 0.0
    jaccard_similarity = float(intersection/union) if union > 0 else 0.0
    overlap_score = intersection/float(min(len(context_set), len(summary_set)) + 1e-11)
    f1_score = float(2 * ((precision * recall)/(precision + recall + 1e-11)))
    
    
    # Bigram Similarity
    answer_bigrams = set(list(ngrams(tokens, 2))) if length > 1 else set()
    context_bigrams = set(list(ngrams(context_tokens, 2))) if len(context_tokens) > 1 else set()
    intersection_bigram = len(answer_bigrams.intersection(context_bigrams))
    union_bigram = len(answer_bigrams.union(context_bigrams))
    precision_bigram = intersection_bigram /len(answer_bigrams) if len(answer_bigrams) > 0 else 0.0
    recall_bigram = intersection_bigram / len(context_bigrams) if len(context_bigrams) > 0 else 0.0
    jaccard_bigram = intersection_bigram/union_bigram if union_bigram > 0 else 0.0
    bigram_overlap = intersection_bigram/float(min(len(context_bigrams), len(answer_bigrams)) + 1e-11)    
    f1_bigram = 2 * ((precision_bigram * recall_bigram)/(precision_bigram + recall_bigram + 1e-11))
    
    # Trigram Similarity
    answer_trigrams = set(list(ngrams(tokens, 3))) if length > 1 else set()
    context_trigrams = set(list(ngrams(context_tokens, 3))) if len(context_tokens) > 1 else set()
    intersection_trigram = len(answer_trigrams.intersection(context_trigrams))
    union_trigram = len(answer_trigrams.union(context_trigrams))
    precision_trigram = intersection_trigram / len(answer_trigrams) if len(answer_trigrams) > 0 else 0.0
    recall_trigram = intersection_trigram / len(context_trigrams) if len(context_trigrams) > 0 else 0.0
    jaccard_trigram = intersection_trigram/union_trigram if union_trigram > 0 else 0.0
    trigram_overlap = intersection_trigram/float(min(len(context_trigrams), len(answer_trigrams)) + 1e-11)    
    f1_trigram = 2 * ((precision_trigram * recall_trigram)/(precision_trigram + recall_trigram + 1e-11))
    
    # Readability
    ASL = float(total_tokens / sentence)
    ASW = float(num_syllables / total_tokens)

    flesch_reading_ease = 206.835 - (1.015 * ASL) - (84.6 * ASW)
    flesch_kincaid = (0.39 * ASL) + (11.8 * ASW) - 15.59
    
    # Automated Reading Index
    characters = len("".join(tokens))
    summary_ari = 4.71 * (characters / total_tokens) + 0.5 * (total_tokens / sentence) - 21.43
    
    # Coleman-Liau Index Readability
    # Where: L is the average number of letters per 100 words, S is the average number of sentences per 100 words
    L = (characters / total_tokens) * 100
    S = (sentence / total_tokens) * 100
    coleman_index = (0.0588 * L) - (0.296 * S) - 15.8
    
    # Organizing features in a dictionary
    features = {
        'length': length,
        'num_chars': len(row['text']),
        "capital_error": capital_error,
        "intersection_bigram": intersection_bigram,
        "union_bigram": union_bigram,
        "jaccard_bigram": jaccard_bigram,
        'recall_bigram': recall_bigram,
        "precision_bigram": precision_bigram,
        "f1_bigram": f1_bigram,
        'bigram_overlap': bigram_overlap,
        'sentence': sentence,
        'avg_sentence_length': avg_sentence_length, 
        'max_sentence_length': max_sentence_length,
        'min_sentence_length':min_sentence_length,
        "std_sentence_length": std_sentence_length,
        "avg_sentence": avg_sentence,
        'avg_unique_sentence':avg_unique_sentence,
        "context_avg_word_len": context_avg_word_len,
        "error": error,
        'unique_words': unique_words,
        'num_entities': num_entities,
        "context_num_entities": context_num_entities,
        'coverage_score': coverage_score,
        'num_overlap_entities': num_overlap_entities,
        'avg_word_len': avg_word_len,
        'intersection_trigram': intersection_trigram,
        'union_trigram': union_trigram,
        'recall_trigram': recall_trigram,
        'precision_trigram': precision_trigram,
        'jaccard_trigram': jaccard_trigram,
        'trigram_overlap': trigram_overlap,
        'f1_trigram': f1_trigram,
        "stops": stops,
        "num_nouns": num_nouns,
        "num_verbs": num_verbs,
        "num_adverbs": num_adverbs,
        "context_nouns": context_nouns,
        "context_stops": context_stops, 
        'context_verbs': context_verbs,
        'context_adverbs': context_adverbs,
        "num_numerical_entities": num_numerical_entities,
        "sentiment_polarity": sentiment_polarity,
        'context_polarity': context_polarity, 
        "nsubj": results['nsubj'],
        "amod": results['amod'],
        "advmod": results['advmod'],
        "xcomp": results['xcomp'],
        "acomp": results['acomp'],
        "past": results["past"],
        "present": results["present"],
        "average_dependency_tree_depth": average_tree_depth,
        "TTR": unique_words / (total_tokens + 1e-8),
        "RTTR": unique_words / (math.sqrt(total_tokens) + 1e-8),
        "CTTR": unique_words / (math.sqrt(total_tokens / 2) + 1e-8),
        # MATTR would require a window-based approach and is thus more involved
        "Herdan's C": math.log(unique_words) / (math.log(total_tokens) + 1e-8),
        "Dugast's U": math.log(total_tokens)**2 / ((math.log(total_tokens) - math.log(unique_words)) + 1e-8),
        "Honoré's H": 100 * math.log(total_tokens) / (1 - once/(unique_words + 1e-8)),
        "Entropy": -sum(p * math.log(p) for p in prob_word),
        "Sichel’s S": twice,
        'named_entity_ratio': named_entity_ratio,
        'context_entity_ratio': context_entity_ratio,
        'flesch_reading_ease': flesch_reading_ease,
        'flesch_kincaid': flesch_kincaid,
        'summary_ari':summary_ari,
        'coleman_index': coleman_index,
        "Simpson’s D": sum((count/total_tokens)**2 for count in word_counts.values()),
        'intersection': intersection,
        "subjectivity": subjectivity,
        "context_subjectivity": context_subjectivity,
        'difference': difference,
        'union': union,
        'overlap_score': overlap_score,
        'recall': recall,
        'precision': precision,
        'f1_score': f1_score,
        'jaccard_similarity': jaccard_similarity,
    }
    
    return features

def compute_similarity(row):
    source_vector = nlp(row['prompt_text']).vector
    answer_vector = nlp(row['text']).vector
    embedding_similarity = cosine_similarity([source_vector], [answer_vector])[0][0]
    euclidean = euclidean_distances([source_vector], [answer_vector])[0][0]
    pearson = np.corrcoef(source_vector.ravel(), answer_vector.ravel())[0, 1]
    
    return pd.Series([embedding_similarity, euclidean, pearson])

def compute_statistical_measures(embedding):
    return np.mean(embedding), np.std(embedding), np.min(embedding), np.max(embedding)

def get_embeddings(row, vectorizer, transformer):
    bag_of_words = transformer.fit_transform(vectorizer.transform([row['lemmas']])).toarray()
    context_words = transformer.fit_transform(vectorizer.transform([row['full_text']])).toarray()
    
    difference = context_words - bag_of_words
    addition = bag_of_words + context_words
    bags_ratio = context_words / (bag_of_words + 1e-13)
        
    word_features = compute_statistical_measures(bag_of_words)
    difference_features = compute_statistical_measures(difference)
    addition_features = compute_statistical_measures(addition)
    ratio_features = compute_statistical_measures(bags_ratio)
    
    similarity = cosine_similarity(context_words, bag_of_words)[0][0]
    distance = euclidean_distances(context_words, bag_of_words)[0][0]
    manhattan_distances = np.sum(np.abs(context_words.ravel() - bag_of_words.ravel()))
    
    return np.hstack((word_features,difference_features,
                      addition_features,ratio_features, 
                      similarity, distance, manhattan_distances
                     ))
    
def vectorize_tokens(train_df, ngram_range=(1,1)):
    vectorizer = CountVectorizer(lowercase=True, ngram_range=ngram_range)
    transformer = TfidfTransformer()
    vectorizer.fit(train_df['lemmas'])
    embed = np.asarray(train_df.apply(get_embeddings, axis=1, args=(vectorizer, transformer)).tolist(), dtype=np.float32) 
    
    return embed

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def preprocess_data(summary_df, prompt_df):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    print('processing text embeddings......')
    train_df = train_df.assign(
      full_text='Title:\n' + train_df['prompt_title'] +
                '\nQuestion:\n' + train_df['prompt_question'] +
                '\nText:\n' + train_df['prompt_text']
      )
    
    
    train_df['lemmas'] = train_df['text'].apply(extract_lemmas)
    train_df['full_text'] = train_df['full_text'].apply(extract_lemmas)
    
    print('Fitting vectorizer')
    embed = vectorize_tokens(train_df, ngram_range=(1,1))
    embed1 = vectorize_tokens(train_df, ngram_range=(2,2))
    
    # processing lemmas
    print('extracting features.....')
    data = pd.DataFrame(train_df.apply(extract_features, axis=1).tolist())
    
    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)
    
    cols = ['summary_punctuation','content']
    cols1 = ['summary_punctuation', 'wording']

    similarity_df = train_df.apply(compute_similarity, axis=1)
    similarity = similarity_df.values
    
    content_bag = np.hstack((embed, embed1, data.values, similarity, train_df[cols].values))
    wording_bag = np.hstack((data.values, similarity, train_df[cols1].values))
    
    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return content_bag, wording_bag, student_ids

In [ ]:
from nltk.corpus import words
english = set(words.words())

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

start_time = time.time()

content_bag, wording_bag, student_ids = preprocess_data(summary_df,prompt_df)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Preprocessed function executed in: {elapsed_time:.2f} seconds")

In [ ]:
def split_data(df):
    X_train,X_test,y_train,y_test = train_test_split(df[:,:-1] ,df[:,-1],test_size=0.1,shuffle=True,random_state=11)
    return X_train,X_test,y_train,y_test

In [ ]:
def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred)**2, axis=0)))

### Train Model

In [ ]:
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

def train_model(df):
    X_train,X_test,y_train,y_test = split_data(df)
    start_time = time.time()

    best_params = {
        'learning_rate': 0.01,
        'n_estimators': 600,
        'max_depth': 11,
        'subsample': 0.6,
        'colsample_bytree': 0.6,
        'num_leaves': 31,
        'objective': 'regression',
        'metric': 'rmse'
    }

    best_models = []

    score = []
    kfold = KFold(n_splits=20, shuffle=True, random_state=64)
    for train_idx, test_idx in tqdm(kfold.split(X_train, y_train), desc='training model'):
        X, y = X_train[train_idx], y_train[train_idx]
        test,test_y = X_train[test_idx], y_train[test_idx]
        pipeline = make_pipeline(
            MinMaxScaler(feature_range=(-10,10)),
            lgb.LGBMRegressor(**best_params, random_state=64)
        )
        
        pipeline.fit(X, y)
        score.append(np.sqrt(mean_squared_error(test_y,pipeline.predict(test))))
        best_models.append(pipeline)

    print(f'Train Mean Score: {np.mean(score):.4f}')
    
    best_scores = []
    for i in range(len(best_models)):
        best_model = best_models[i]
        y_preds = best_model.predict(X_test)
        best_scores.append(np.sqrt(mean_squared_error(y_test,y_preds)))
    print(f'Evaluation Mean Score: {np.mean(best_scores):.4f}')
    
    best_model_idx = np.argmin(best_scores)
    best_model = best_models[best_model_idx]
    
    y_preds = best_model.predict(X_test)
    score = np.sqrt(mean_squared_error(y_test,y_preds))
    
    end_time = time.time()
    elapsed_time = end_time - start_time 
    print(f"Preprocessed function executed in: {elapsed_time:.2f} seconds")
    print(f'Best Model Score: {score:.4f}')
    
    return best_model, y_test, y_preds

In [ ]:
content_model, content_test, content_preds = train_model(content_bag)

In [ ]:
wording_model, wording_test, wording_preds  = train_model(wording_bag)

In [ ]:
y_test = np.hstack((content_test.reshape((-1,1)), wording_test.reshape((-1,1))))
preds = np.hstack((content_preds.reshape((-1,1)), wording_preds.reshape((-1,1))))

In [ ]:
mcrmse(y_test, preds)

#### 0.4700925199890056

### Preprocessing for text data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def preprocess_data(summary_df, prompt_df):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    print('processing text embeddings......')
    train_df = train_df.assign(
      full_text='Title:\n' + train_df['prompt_title'] +
                '\nQuestion:\n' + train_df['prompt_question'] +
                '\nText:\n' + train_df['prompt_text']
      )
    
    train_df['lemmas'] = train_df['text'].apply(extract_lemmas)
    train_df['full_text'] = train_df['full_text'].apply(extract_lemmas)
    
    print('Fitting vectorizer')
    embed = vectorize_tokens(train_df, ngram_range=(1,1))
    embed1 = vectorize_tokens(train_df, ngram_range=(2,2))
    
    # processing lemmas
    print('extracting features.....')
    data = pd.DataFrame(train_df.apply(extract_features, axis=1).tolist())
    
    print('processing features.....')
    # Process other columns
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)
    
    cols = ['summary_punctuation']

    similarity_df = train_df.apply(compute_similarity, axis=1)
    similarity = similarity_df.values
    
    content_bag = np.hstack((embed, embed1, data.values, similarity, train_df[cols].values))
    wording_bag = np.hstack((data.values, similarity, train_df[cols].values))
    
    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return content_bag, wording_bag, student_ids

In [ ]:
content, wording,ids = preprocess_data(test_summary, test_prompts)

In [ ]:
cont_preds = content_model.predict(content).reshape((-1,1))
word_preds = wording_model.predict(wording).reshape((-1,1))
y_preds = np.hstack((cont_preds, word_preds))

In [ ]:
df_test = pd.DataFrame(y_preds,columns=['content','wording'],index=ids).reset_index()
df_test = df_test.rename({'index':'student_id'},axis=1)

In [ ]:
def is_valid_float(x):
    return isinstance(x, float) and x == x  # This checks that x is not NaN since NaN != NaN in Python.

cols_to_check = ['wording', 'content']
df_test[cols_to_check] = df_test[cols_to_check].applymap(lambda x: x if is_valid_float(x) else 0.0)

df_test.to_csv("submission.csv", index=False)